In [1]:
import tables
import h5py
import numpy as np
import sys
from pyhdf.SD import SD, SDC
import pandas as pd
import time
import calendar
import matplotlib.pyplot as plt
import datetime
import os
import openturns as ot

## File Parsing

**Sample File:** Re-gridded MODIS AOD 2010, Days 183 through 273

In [2]:
tot = np.array(pd.read_hdf("regridded_2010.h5"))

In [3]:
tot = pd.DataFrame(tot)

In [4]:
tot = tot.iloc[(int)(len(tot) / 2):(int)(3 * len(tot) / 4)]

In [5]:
tot

,0,1,2,3,4,5,6,7,8,9,...,14390,14391,14392,14393,14394,14395,14396,14397,14398,14399
182,79.250000,0.0,137.000000,96.5,0.000000,0.0,0.0,0.00,0.000000,51.000000,...,93.0,157.0,116.5,87.500000,136.0,146.5,120.000000,163.5,125.5,54.000000
183,0.000000,0.0,0.000000,0.0,0.000000,0.0,96.0,103.50,103.333333,110.666667,...,54.0,0.0,81.0,0.000000,76.0,104.0,75.666667,79.0,52.0,0.000000
184,121.500000,0.0,150.500000,0.0,190.000000,0.0,0.0,65.00,27.500000,0.000000,...,-37.0,96.0,77.5,0.000000,109.0,115.0,123.000000,133.5,173.0,60.000000
185,114.500000,122.0,148.666667,169.0,181.666667,0.0,82.0,96.00,131.000000,152.000000,...,98.0,104.0,106.5,135.333333,136.0,124.0,88.500000,98.5,94.0,78.500000
186,0.000000,0.0,0.000000,0.0,0.000000,0.0,48.0,62.00,57.000000,57.000000,...,171.0,0.0,133.0,0.000000,0.0,0.0,0.000000,198.0,0.0,137.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,...,0.0,167.0,0.0,133.500000,0.0,0.0,72.000000,0.0,53.0,0.000000
269,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,77.00,0.000000,88.000000,...,0.0,0.0,115.0,110.333333,74.0,107.0,99.000000,160.5,137.0,60.666667
270,0.000000,0.0,0.000000,0.0,0.000000,0.0,152.0,123.00,95.500000,91.500000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
271,54.333333,72.5,55.000000,55.0,0.000000,0.0,0.0,27.75,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,164.000000,0.0,181.0,139.000000


In [6]:
len(tot)

91

## Interpolation

In [2]:
fin = []

for day in range(len(tot)):

    # Data Prep
    krig_data = tot.iloc[day]
    krig_data_train = krig_data[krig_data != 0] #gets values
    data_indices = krig_data_train.reset_index().index
    krig_data_skip = None
    print("check 1", len(krig_data_train))
    if (len(krig_data_train) < 3):
        fin.append(fin[-1])
        print(i)
        continue
    if (6000 >= len(krig_data_train) > 3000):
        mask = np.where(data_indices % 2 != 0, False, True)
        krig_data_skip = krig_data_train[~mask]
        krig_data_train = krig_data_train[mask]
    elif len(krig_data_train) > 6000:
        mask = np.where(data_indices % 3 != 0, False, True)
        krig_data_skip = krig_data_train[~mask]
        krig_data_train = krig_data_train[mask]
    print("check 2", len(krig_data_train))
    krig_data_test = krig_data[krig_data == 0]
    lons = [krig_data_train.index[i] % 120 for i in range(len(krig_data_train))]
    lats = [(krig_data_train.index[i] - (krig_data_train.index[i] % 120)) / 120 for i in range(len(krig_data_train))]
    c = [np.array(i) for i in zip(lats, lons)]
    d = [[krig_data_train.iloc[i]] for i in range(len(krig_data_train))]
    coordinates_train = ot.Sample(c)
    aod_train = ot.Sample(d)

    # Training with openturns package
    inputDimension = 2
    basis = ot.ConstantBasisFactory(inputDimension).build()
    covarianceModel = ot.SquaredExponential([1.]*inputDimension, [1.0])
    algo = ot.KrigingAlgorithm(coordinates_train, aod_train, covarianceModel, basis)
    algo.run()
    result = algo.getResult()
    krigingMetamodel = result.getMetaModel()

    # Prediction
    lons_test = [krig_data_test.index[i] % 120 for i in range(len(krig_data_test))]
    lats_test = [(krig_data_test.index[i] - (krig_data_test.index[i] % 120)) / 120 for i in range(len(krig_data_test))]
    c = [np.array(i) for i in zip(lats_test, lons_test)]
    kriged_data_pred = krigingMetamodel(c)
    kriged_data_pred = pd.Series(np.array(kriged_data_pred).reshape(-1))
    kriged_map_arr = np.zeros((120, 120))

    for i in range(len(krig_data_test.index)):
        kriged_map_arr[(int)((krig_data_test.index[i] - (krig_data_test.index[i] % 120)) / 120), (int)(krig_data_test.index[i] % 120)] = kriged_data_pred[i]

    for i in range(len(krig_data_train.index)):
        kriged_map_arr[(int)((krig_data_train.index[i] - (krig_data_train.index[i] % 120)) / 120), (int)(krig_data_train.index[i] % 120)] = krig_data_train[krig_data_train.index[i]]
    
    if (krig_data_skip is not None):
        for i in range(len(krig_data_skip.index)):
            kriged_map_arr[(int)((krig_data_skip.index[i] - (krig_data_skip.index[i] % 120)) / 120), (int)(krig_data_skip.index[i] % 120)] = krig_data_skip[krig_data_skip.index[i]]
    
    fin.append(kriged_map_arr)
    print(day)

NameError: name 'tot' is not defined

## File Export

In [8]:
pd.DataFrame(np.array(fin).reshape(len(tot), 120*120)).to_hdf("aod_interp_2010_3.h5", key="df")